<a href="https://www.kaggle.com/code/andymejia/breastcancer-data?scriptVersionId=144790957" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
df = pd.read_csv('/kaggle/input/breast-cancer-dataset/breast-cancer.csv')

In [ ]:
pd.read_csv('/kaggle/input/breast-cancer-dataset/breast-cancer.csv')

In [ ]:
df.keys()

In [ ]:
df.describe()

In [ ]:
#id = ID number of the image data
#diagnosis = M(malignant), B(benign)
#radius_mean = Mean of distance from center to points of the perimeter
#texture mean = STD of grey-scale  values
#preiemnter_mean = Mean size of the core tumor
#area_mean = mean area of the tumor
#smoothness_mean = Mean local variation in radius lengths
#compactness_mean = Mean of perimeter^2/area -1.0
#concavity_mean = Mean of severity of concave portions of the contour
#concave points_mean = Mean number of concave portions of the contour

In [ ]:
#Count of the occurances of each  class
diagnosis_counts = df['diagnosis'].value_counts()


plt.bar(diagnosis_counts.index, diagnosis_counts.values)

plt.xlabel('Diagnosis')
plt.ylabel('Count')
plt.title('Distribution of Target Variable (Diagnosis)')

plt.show()

In [ ]:
# Set up the matplotlib figure
f, axes = plt.subplots(2, 2, figsize=(14, 14), sharex=False)

# Plot a histogram for 'radius_mean'
sns.histplot(df['radius_mean'], color="skyblue", ax=axes[0, 0], kde=True)

# Plot a histogram for 'texture_mean'
sns.histplot(df['texture_mean'], color="olive", ax=axes[0, 1], kde=True)

# Plot a histogram for 'perimeter_mean'
sns.histplot(df['perimeter_mean'], color="gold", ax=axes[1, 0], kde=True)

# Plot a histogram for 'area_mean'
sns.histplot(df['area_mean'], color="teal", ax=axes[1, 1], kde=True)

# Set the titles for each subplot
axes[0, 0].set_title('Radius Mean')
axes[0, 1].set_title('Texture Mean')
axes[1, 0].set_title('Perimeter Mean')
axes[1, 1].set_title('Area Mean')

# Display the plots
plt.tight_layout()
plt.show()

In [ ]:
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
plt.figure(figsize=(15, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
print(correlation_matrix)

In [ ]:
X = df.drop('diagnosis',axis=1)
y = df['diagnosis']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,   random_state=42)

In [ ]:
#Setting up the logisticRegression
model = LogisticRegression(max_iter=10000, random_state=42)

In [ ]:
from sklearn.feature_selection import RFE

# Create an RFE selector object using the logistic regression model as the estimator
selector = RFE(model, n_features_to_select=5, step=1)

# Fit the model
selector = selector.fit(X_train, y_train)

In [ ]:
selected_features = X_train.columns[selector.support_]
print("Selected Features:", selected_features)

In [ ]:
feature_ranking = selector.ranking_
print('Feature Ranking', feature_ranking)

In [ ]:
#Building a model with selected features.
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

model.fit(X_train_selected, y_train)

In [ ]:
y_pred = model.predict(X_test_selected)

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Classification Report: ", classification_report(y_test, y_pred))

In [ ]:
df = df.drop(columns=['id'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rfe = RandomForestClassifier(random_state=42)

In [ ]:
model_rfe.fit(X_train, y_train)

In [ ]:
importances = model_rfe.feature_importances_

In [ ]:
feature_importances = pd.Series(importances, index=X_train.columns)
sorted_features = feature_importances.sort_values(ascending=False)

In [ ]:
selected_features = sorted_features.head(5).index.tolist()

In [ ]:
print("sorted_features:, selected_features")

In [ ]:
print("sorted_features:", sorted_features)
print("selected_features:", selected_features)

In [ ]:
# Selected features
selected_features = ['perimeter_worst', 'area_worst', 'concave points_worst', 'radius_worst', 'concave points_mean']

# Subset the data
X_selected = X[selected_features]

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Split the data
X_train_selected, X_test_selected, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_selected, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test_selected)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)